In [ ]:
%pip install math_verify

In [ ]:
import os
import re
import torch
from datasets import load_dataset
from tqdm.auto import tqdm
from math_verify import verify, parse
from transformers import (
    AutoTokenizer,
    AutoModelForCausalLM,
    TrainingArguments,
    Trainer
)

In [ ]:
def extract_xml_answer(text: str) -> str:
    answer = text.split("<answer>")[-1]
    answer = answer.split("</answer>")[0]
    return answer.strip()

In [ ]:
def extract_boxed_answer(text: str) -> str:
    boxed_pattern = r"\\boxed\{((?:[^{}]|\{[^{}]*\})*)\}"
    matches = re.findall(boxed_pattern, text)
    
    if matches:
        return matches[-1]
    
    return ""

In [ ]:
SYSTEM_PROMPT = """
Respond in the following format:
<reasoning>
...
</reasoning>
<answer>
...
</answer>
"""
MODEL_PATH = '/home/jupyter/datasphere/project/check_sft'
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [ ]:
DEVICE

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("Qwen/Qwen2.5-1.5B")
model = AutoModelForCausalLM.from_pretrained("./check_sft")

In [ ]:
model = model.to(DEVICE)

In [ ]:
test_dataset = load_dataset("nlile/hendrycks-MATH-benchmark", split="test")
sample = test_dataset.select(range(5))

correct_answers = 0
total_answers = 0

decoded = {'problem': [], 'reasoning': [], 'answer': [], 'true_answer': []}

for example in tqdm(sample, desc="Оценка Success Rate с чекпоинта"):
    problem_text = example["problem"]
    ground_truth_answer = example["answer"]
    
    decoded['true_answer'].append(ground_truth_answer)    
    decoded['problem'].append(problem_text)

    if ground_truth_answer is None:
        continue
    total_answers += 1

    prompt = SYSTEM_PROMPT + " Problem: " + problem_text
    inputs = tokenizer(prompt, return_tensors="pt").to(DEVICE)

    outputs = model.generate(
        **inputs,
        max_new_tokens=1024,
        temperature=0.01,
        top_p=1,
        do_sample=True,
        pad_token_id=tokenizer.eos_token_id
    )
    
    generated_output = tokenizer.decode(outputs[0], skip_special_tokens=True)
    decoded['reasoning'].append(generated_output)
    model_answer_str = extract_boxed_answer(generated_output)
    decoded['answer'].append(model_answer_str)

    if verify(parse(model_answer_str), parse(ground_truth_answer), float_rounding=2):
        correct_answers += 1

if total_answers > 0:
    success_rate = (correct_answers / total_answers) * 100
    print("\n" + "="*50)
    print("="*50)
    print(f"✅ Верных ответов: {correct_answers} из {total_answers}")
    print(f"📈 Итоговый Success Rate: {success_rate:.2f}%")
    print("="*50)
else:
    print("Не найдено примеров с эталонным ответом для оценки.")

In [ ]:
decoded